<a href="https://colab.research.google.com/github/Baldros/Llama3.1-fine-tuning-/blob/main/ModeloFineTunado(Llama3_1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Apresentação:

Dado todo o processo de fine-tuning do modelo Llama3.1 utilizando a [unsloth](https://docs.unsloth.ai/), feito em outro [notebook](https://colab.research.google.com/drive/1G2czQksWDOhUtIRKHQi__kIj5Mlq5ooj?usp=sharing), o objetivo deste código é explorar o processo de utilização do modelo.

In [ ]:
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-7k4jkhax/unsloth_e39e8e6da6f94e1e9c30209bb078211b
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-7k4jkhax/unsloth_e39e8e6da6f94e1e9c30209bb078211b
  Resolved https://github.com/unslothai/unsloth.git to commit e4c8ceacb3fca634f78e662873a01c37678fcb3e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 111.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 110.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 11.5 MB/s eta 0:00:00
   

# Descompactando o Arquivo:

In [ ]:
import zipfile
import os

In [ ]:
def descompactar_zip(caminho_arquivo_zip, destino_pasta):
    """Descompacta um arquivo ZIP para a pasta especificada."""
    try:
        # Verifica se o arquivo ZIP existe
        if not os.path.isfile(caminho_arquivo_zip):
            print(f"O arquivo ZIP '{caminho_arquivo_zip}' não foi encontrado.")
            return

        # Cria a pasta de destino se não existir
        if not os.path.exists(destino_pasta):
            os.makedirs(destino_pasta)

        # Descompacta o arquivo ZIP
        with zipfile.ZipFile(caminho_arquivo_zip, 'r') as zip_ref:
            zip_ref.extractall(destino_pasta)
            print(f"Arquivo ZIP descompactado com sucesso em '{destino_pasta}'.")

    except Exception as e:
        print(f"Ocorreu um erro ao descompactar o arquivo ZIP: {e}")

In [ ]:
# Descompactando:
descompactar_zip('/content/llama3.1 (fine-tuning).zip','/content')

Arquivo ZIP descompactado com sucesso em '/content'.


# Utilizando Modelo:

In [ ]:
# Importações:
from unsloth import FastLanguageModel
from transformers import TextStreamer
import numpy as np

In [ ]:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

In [ ]:
# Instanciando Modelo:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "/content/llama3.1 (fine-tuning)",
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True
)

==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.0.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
# Abilitando aceleração de inferência (2x)
FastLanguageModel.for_inference(model)

In [ ]:
def prompt(max_length=5000):
  entrada = input('Prompt:\n')

  # Tokenizar o texto
  inputs = tokenizer(entrada, return_tensors="pt")

  # Gerar texto
  outputs = model.generate(**inputs, max_length=max_length)

  # Decodificar a resposta gerada
  generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

  i = len(entrada)
  return generated_text[i:]

In [ ]:
resposta = prompt()

Prompt:
Quais são os direitos de um estagiário? 


In [ ]:
print(resposta)

1. O estagiário tem direito a uma vaga em uma instituição de saúde no trabalho. 2. O estagiário tem direito a uma remuneração justa e adequada por seus serviços. 3. O estagiário tem direito a uma relação de trabalho seguro e saudável. 4. O estagiário tem direito a uma formação continua em seus serviços. 5. O estagiário tem direito a uma proteção contra discriminações e assédios. 6. O estagiário tem direito a uma representação adequada dos interesses trabalhistas.
  1. O estagiário tem direito a uma vaga em uma instituição de saúde no trabalho: Essa vaga é fundamental para garantir que os estagiários tenham acesso a serviços de saúde e segurança no local de trabalho, promovendo uma cultura de saúde e segurança para todos os trabalhadores. 2. O estagiário tem direito a uma remuneração justa e adequada por seus serviços: A remun thời giana justa reconhece o valor dos serviços dos estagiários e contribui.filePath para a qualidade dos serviços oferecidos. 3. O estagiário tem direito a uma r